In [2]:
import pandas as pd
import numpy as np

In [5]:
a = [0, 3, 4, 5, 6]
b = 2
schd_dep_after = np.array(a)
schd_dep_after = schd_dep_after[schd_dep_after>b]
np.sort(schd_dep_after)[:2].tolist()

[3, 4]